# Creating and plotting elemental EELS maps

In this tutorial, elemental maps of copper and zinc will be created from electron energy loss spectra (EELS). The data is recorded using a sample of copper and zinc oxide deposited on carbon nanotubes. The particles are likely to be very small, and the carbon is not completely covered. In this sample, the ratio of Zn and Cu is 3:1, and approximately 80 wt% of the sample is carbon. Due to the small amount of copper and zinc, the signal strength is low. To improve the signal strength without lowering the spatial resolution, the spectra were recorded with a high energy dispersion, resulting in low energy resolution. This means that any fine structure is not visible.

For more information about the material system and how the results of this processing can be used, see the paper: https://doi.org/10.1016/j.cattod.2019.02.045

This notebook requires `HyperSpy` 2.0 or higher. It also requires `eXSpy` which is a library which contains all of HyperSpy's EELS and EDX functionality.

The main objective is to show how the functionalities of HyperSpy and eXSpy can be used to create relative elemental maps from EEL spectra.



### Changes

* 2017/09/27: Initial version by Ida Hjorth
* 2019/11/14: Update to HyperSpy 1.5, and minor improvements to text by Magnus Nord
* 2024/3/16: Update to work with HyperSpy 2.0, by Magnus Nord

### Table of contents

1. <a href='#load'> Load data</a> 
2. <a href='#fitting'>Fitting Cu and Zn Hartree-Slater cross sections</a>
3. <a href='#save_restore'>Saving and restoring the model</a>
4. <a href='#plotting'>Creating elemental maps from the model</a>

# <a id='load'></a>1. Load data

In [ ]:
%matplotlib widget
import numpy as np
import hyperspy.api as hs

In [ ]:
s_eels = hs.load('datasets/CuZn_EELS_mapping_tutorial.hspy')

Because of the overlap of the Cu (around 930 eV) and Zn edges (around 1040 eV), it is very difficult to estimate background for the Zn edge. A solution to this problem is to fit the background and edges as shown in this notebook.

Visualise the data using `plot()`, and navigate to a region where both edges are visible. For example position `(32, 14)`. Move the navigator in the `EELS Spectrum Image Navigator` either by:

- left-click and drag the red "box" in the `EELS Spectrum Image Navigator`
- select the `EELS Spectrum Image Navigator` window/figure and use the `Ctrl` + `Arrow` keys on your keyboard,
- or hold the `Shift` key on your keyboard and left click on the location you want to navigate to.

To compare two spectra from different positions, select either one of the figures and press the `E` key on your keyboard. This will add an extra blue navigator "box", which is moved by left-click + dragging it.

In [ ]:
s_eels.plot()

# <a id='fitting'></a>2. Fitting Cu and Zn components to the spectrum

In the model-based approach, the low-loss signal can be convolved with the high-loss signal such that the energy spread of the electron beam and plural scattering due to the bulk plasmon are taken into account. This will lead to a better fit of the model to the experimental data. In addition, the experimental zero loss peak can be used to precisely calibrate the energy scale. 

In this example the low-loss signal was not recorded. However, since the sample is fairly thin, the effect of plural scattering is negligible.

------

**Note**: this will download a set of open source **Generalised Oscillator Strengths (GOS)** , which was computed and packaged by Leonhard Segger, Giulio Guzzinati and Helmut Kohl. For more information about the GOS files, see the [Zenodo deposit](https://zenodo.org/doi/10.5281/zenodo.6599070). For more information about how these were computed, and the code used to compute them, see https://github.com/Br0Fi/goscalc.

Now, we will make a model containing a power law background and the Cu and Zn cross sections. To do this, we first have to set the experimental parameters.

In [ ]:
s_eels.set_microscope_parameters(beam_energy=200, convergence_angle=13.33, collection_angle=55.28) 
s_eels.add_elements(('Cu', 'Zn')) 
m = s_eels.create_model()

Both the power law background and core less edge components (`EELSCLEdge`) has automatically been added.

In [ ]:
m.components

Due to the large intensity variations across the dataset, the different features are fitted one by one within limited energy ranges. After having run the fitting for the individual features, all of them will be fitted at the same time.

Here, we use the `model.fit_component` method for doing the initial fitting of the individual features. It works by passing the component we want to fit, and the energy range where we want to do this fitting through the `signal_range` parameter. In addition, we set `only_current=False` to fit all the probe positions.

**Tip:** you can use docstrings to learn how functions work. These are accessed by having a question mark behind the function, for example `m.fit_component?`. If you're using JupyterLab, you can also press the `Shift` + `Tab` keys on your keyboard while the function is selected in a code cell.

We start with the power law component, and fit it from the start of the spectrum (700 eV) to right before the first (Cu) core loss edge (900 eV).

In [ ]:
m.fit_component(m.components.PowerLaw, signal_range=(700, 900), only_current=False)

Now, we visualise the model to see how well the fitting went.

The model consist of several components, so we use `plot_components=True` to see the individual components. Here, the red dots are the experimental data, the blue line is the whole model, and the green line is the power law we just fitted. There are also several other components shown in the figure: the Cu and Zn core loss edges. When added, they start with an initial non-zero value, so don't worry that they currently do not fit very well with the data.

In [ ]:
m.plot(plot_components=True)

Then we use `fit_component` again, to fit the first of the core loss edges: Cu. Here, we want the energy range which contains just power law (which we already fitted), and the Cu-L$_{2,3}$ edges. For example from 950 to 1000 eV. Again, we use `only_current=False` to fit all the probe positions.

In [ ]:
m.fit_component(m.components.Cu_L3, signal_range=(950, 1000), only_current=False)

Then we repeat this for Zn-L$_{2,3}$, for signal range 1050 to 1150 eV.

In [ ]:
m.fit_component(m.components.Zn_L3, signal_range=(1050, 1150), only_current=False)

Lastly, we force both the Cu- and Zn-edges to always have positive intensity, as negative composition is not very physical.

This sets the lower bound for the intensity of these edges to 0.

In [ ]:
m.set_all_edges_intensities_positive()

Finally, we fit all the components at the same time.

In [ ]:
m.multifit()

Let's see the results. Go to the region around `(32, 14)`, where there clearly is copper or zinc of some kind.

In [ ]:
m.plot(plot_components=True)

### Simple visualization

Visualizing the intensity of the Cu and Zn edges. **Note**: as the L$_3$, L$_2$ and L$_1$ are connected in the fitting procedure, we only have to get one of them to visualize the relative amount of Cu or Zn.

In [ ]:
m.components.Zn_L3.intensity.plot()

In [ ]:
m.components.Cu_L3.intensity.plot()

To more easily compare them, use the `hs.plot.plot_images` function. To get an overlay, where the two intensities are shown with different colors in the same plot, use `overlay=True`.

In [ ]:
hs.plot.plot_images([m.components.Cu_L3.intensity.as_signal(), m.components.Zn_L3.intensity.as_signal()])

# 3. <a id='save_restore'></a>Saving and restoring the models

As the fitting process can be slow, saving the models can be a good idea. 

In [ ]:
m.save('model.hspy', overwrite=True)

This results in a file called `model.hspy`, which can be loaded and restored.

In [ ]:
mr = hs.load('model.hspy')
mr = mr.models.restore('a')

In [ ]:
mr.plot()

# 4. <a id='plotting'></a>Making publication quality elemental maps

In this section, elemental maps will be plotted from the intensity of the model of the Cu-L$_3$ and Zn-L$_3$ components saved above. The HAADF image will also be plotted, for comparison with the elemental distribution.

**Note:** as the components L$_3$, L$_2$ and L$_1$ are connected during the fitting procedure, we only need to visualize one of them to get a relative elemental map.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [ ]:
m = hs.load('model.hspy')
mr = m.models.restore('a')

In [ ]:
s_copper = mr.components.Cu_L3.intensity.as_signal()
s_zinc = mr.components.Zn_L3.intensity.as_signal()
s_haadf = hs.load('datasets/CuZn_HAADF.hspy').as_signal2D((0, 1))

`GridSpec` is a convenient function to use when creating several subplots. We create axes for the elemental maps, HAADF signal as well as the colorbars that will be used indicate the numerical intensity.

For more information on how to use `GridSpec`, you can access the docstring by running `gridspec.GridSpec?` in a cell.  (Note the question mark)

In [ ]:
fig = plt.figure(figsize=(5.5, 2.7))
gs = gridspec.GridSpec(30, 3)
ax_cu = fig.add_subplot(gs[0:-2, 0])
ax_zn = fig.add_subplot(gs[0:-2, 1])
ax_haadf = fig.add_subplot(gs[0:-2, 2])
cbar_cu =  fig.add_subplot(gs[-1, 0]) # Colorbars are much thinner than the map axes (1/30 of the height of the image)
cbar_zn =  fig.add_subplot(gs[-1, 1])
cbar_haadf =  fig.add_subplot(gs[-1, 2])

The signals can be plotted by using `imshow`.

In [ ]:
cax_cu = ax_cu.imshow(
    s_copper.data,
    interpolation='nearest',
    origin='lower',
    extent=s_copper.axes_manager.signal_extent,
)

In [ ]:
cax_zn = ax_zn.imshow(
    s_zinc.data,
    interpolation='nearest',
    origin='lower',
    extent=s_zinc.axes_manager.signal_extent,
)

NumPy functions, such as `flipud`, `fliplr` and `rot90` can also be used to give the image the correct orientation. Here,`flipud` is used to give the HAADF image the same orientation as the elemental maps.

In [ ]:
ax_haadf.imshow(
    np.flipud(s_haadf.data),
    interpolation='nearest',
    origin='upper',
    extent=s_haadf.axes_manager.signal_extent,
)

Disable axis ticks

In [ ]:
ax_haadf.set_xticks([])
ax_cu.set_xticks([])
ax_zn.set_xticks([])
ax_zn.set_yticks([])
ax_cu.set_yticks([])
ax_haadf.set_yticks([])

Scalebars can be added by using `AnchoredSizeBar`. 

In [ ]:
fontprops = fm.FontProperties(size=18)
scalebar0 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1, # length of bar, label, loc
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
scalebar1 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1,
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
scalebar2 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1,
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
ax_cu.add_artist(scalebar0)
ax_zn.add_artist(scalebar1)
ax_haadf.add_artist(scalebar2)

We add labels to indicate what is shown in each image.

In [ ]:
ax_cu.text(0.05, 0.05, 'Cu', color='white', size=18, transform=ax_cu.transAxes)
ax_zn.text(0.05, 0.05, 'Zn', color='white', size=18, transform=ax_zn.transAxes)
ax_haadf.text(0.05, 0.05, 'HAADF', color='white', size=18, transform=ax_haadf.transAxes)

Then colorbars are added. 

In [ ]:
cb_zn = fig.colorbar(ax_zn.images[0], cax=cbar_zn, extend='both', orientation='horizontal', label="Relative Zn, [a.u.]")
cb_cu = fig.colorbar(ax_cu.images[0], cax=cbar_cu,extend='both', orientation='horizontal', label="Relative Cu, [a.u.]")
cb_haadf = fig.colorbar(ax_haadf.images[0], cax=cbar_haadf, extend='both', orientation='horizontal', label="HAADF, [a.u.]")

In [ ]:
cb_cu.set_ticks([0, 5, 10, 15])
cb_zn.set_ticks([0, 5, 10, 15, 20, 25])
cb_haadf.set_ticks([0, 1000, 2000])

The color schemes `viridis`, `inferno`, `plasma` and `magma` are nice to use as they are grayscale compatible, perceptually uniform and colorblind-proof. For more info on these color maps, see https://www.youtube.com/watch?v=xAoljeRJ3lU

In [ ]:
cax_zn.set_clim(vmin=0, vmax=25)
cax_zn.set_cmap('inferno')
cax_cu.set_clim(vmin=0, vmax=15)
cax_cu.set_cmap('viridis')

In [ ]:
fig.subplots_adjust(left=0, bottom=0.15, right=1, top=0.98)

Saving the matplotlib figure object as a png-file. Here, the resolution (via dots per inch, `dpi`) is set to a high value: 300. 

In [ ]:
fig.savefig("elemental_map.png", dpi=300)

We can see the end result inside the Notebook by using `plt.show()`

In [ ]:
plt.show()